In [ ]:
from pathlib import Path
TSV_PATH = Path("/Users/joregan/Playing/hsi/annotations/word_annotations/main")

In [40]:
import json

with open("/Users/joregan/Playing/hsi/annotations/all_meta.json") as inf:
    data = json.load(inf)

In [45]:
data["12"]

{'recording_id': 'hsi_3_0715_209_008',
 'segment_id': '42',
 'phrase': 'the Rug',
 'utterance': 'Did you move the Rug like in front of the...',
 'object_id': 'Rug',
 'object_name': 'Rug',
 'frame_number': 2756,
 'image_paths': {'color': 'images/color/hsi_3_0715_209_008_42_color.png',
  'depth': 'images/depth/hsi_3_0715_209_008_42_depth.png',
  'overlay': 'images/overlay/hsi_3_0715_209_008_42_overlay.png',
  'mask': 'images/masks/hsi_3_0715_209_008_42_mask.png'},
 'topic': 'Rug',
 'timing': {'phrase_start': 183.735,
  'phrase_end': 184.295,
  'utterance_start': 183.5558059638532,
  'utterance_end': 195.32878031395185},
 'meta_id': 12}

In [54]:
allsegments = {}

for item in data:
    if not data[item]["recording_id"] in allsegments:
        allsegments[data[item]["recording_id"]] = []
    allsegments[data[item]["recording_id"]].append(data[item]["segment_id"])

for seg in allsegments:
    allsegments[seg].sort(key =lambda x: int(x))

In [8]:
data["12"].get("high_level", {}).get("current_topic", "room")

'FloorLamp_82e14abb'

In [18]:
item_keys = list(data.keys()).index("12")

In [19]:
item_keys

12

In [37]:
def get_topic_context(segments, segment_id, size=None, keep_topic=True):
    current_topic = segments[segment_id].get("high_level", {}).get("current_topic", "room")

    item_keys = list(data.keys())
    index = item_keys.index(segment_id)
    print(index)
    if size is None:
        start = 0
    else:
        start = index - size
    ctx_range = item_keys[start:index]

    if len(ctx_range) < size:
        if int(segment_id) <= size:
            pass
        else:
            print(f"Warning: size of {size} cannot be satisfied: {range}")
    
    topics = [segments[x].get("high_level", {}).get("current_topic", "room") for x in ctx_range]
    if keep_topic:
        tmp = []
        for p in zip(ctx_range, topics):
            print(p, segments[p[0]])
            if p[1] == current_topic:
                tmp.append(segments[p[0]]["snippet"])
            else:
                tmp.append(None)
        print(tmp)

In [38]:
get_topic_context(data, "12", 5)

12
('7', 'pet bed') {'general': {'id': 7, 'start': 52.069, 'end': 64.335}, 'high_level': {'current_topic': 'pet bed', 'topic_duration_id': 0}, 'low_level': {'resolved_references': {}}}
('8', 'pet bed') {'general': {'id': 8, 'start': 65.27499999999999, 'end': 74.84}, 'high_level': {'current_topic': 'pet bed', 'topic_duration_id': 1}, 'low_level': {'resolved_references': {'here': 'corner1', 'that': 'corner2'}}}
('9', 'pet bed') {'general': {'id': 9, 'start': 75.44, 'end': 80.146}, 'high_level': {'current_topic': 'pet bed', 'topic_duration_id': 2}, 'low_level': {'resolved_references': {'here': ['corner1']}}}
('10', 'pet bed') {'general': {'id': 10, 'start': 80.386, 'end': 81.888}, 'high_level': {'current_topic': 'pet bed', 'topic_duration_id': 3}, 'low_level': {'resolved_references': {'the windows': 'window'}}}
('11', 'FloorLamp_82e14abb') {'general': {'id': 11, 'start': 103.302, 'end': 107.364}, 'high_level': {'current_topic': 'FloorLamp_82e14abb', 'topic_duration_id': 0}, 'low_level': {

KeyError: 'snippet'